<a href="https://colab.research.google.com/github/marekpiotradamczyk/ml_uwr_22/blob/main/kmeans_deep_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How it differs from the default solution? 

More smaller patches and denser patches extraction from images.

    PATCH_SIZE = 4
    patch_num  = 5000000
    STRIDE     = 2
    
Also, more clustsers:

    kroot = 16
    k = kroot * kroot
    
Note that it takes significant time to execute all steps

# Load data & default imports

In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import scipy.stats as sstats
import multiprocessing as mp
from sklearn import datasets
import sklearn.linear_model
from tqdm.auto import tqdm
from matplotlib import animation, pyplot, rc
import matplotlib.pyplot as plt
import httpimport
from os.path import join
import os.path
from PIL import Image

from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np

from sklearn.feature_extraction import image
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances

In [2]:
!pip install -q gdown httpimport
![ -e cifar.npz ] || gdown 'https://drive.google.com/uc?id=1oBzZdtg2zNTPGhbRy6DQ_wrf5L5OAhNR' -O cifar.npz

In [3]:
with np.load('cifar.npz') as data:
    cifar_train_data = data['train_data']
    cifar_train_labels = data['train_labels']
    cifar_test_data = data['test_data']
    cifar_test_labels = data['test_labels']


def exists(file_path):
    return os.path.isfile(file_path)

In [4]:
X_trn = cifar_train_data
y_trn = cifar_train_labels
X_tst = cifar_test_data
y_tst = cifar_test_labels

# Deep Features

## find important patterns in patches

In [24]:
PATCH_SIZE = 6
patch_num  = 100000
STRIDE     = 8
kroot = 16
k = kroot **2

In [40]:
class clf():
      
    def __init__():
        pass
    
    def contrast(image):
        return (image-image.min())/(image.max() - image.min())

    def normalize_patch(patch, eps=10):
        return (patch - patch.mean())/np.sqrt(patch.var() + eps)

    def whiten(X):
        X_norm = (X - X.mean(axis=0))/X.std(axis=0)
        cov = np.cov(X_norm, rowvar=False) 
        U,S,V = np.linalg.svd(cov)

        X_zca = U.dot(np.diag(1.0/np.sqrt(S + 0.1))).dot(U.T).dot(X_norm.T).T
        return X_zca

    def centroids(data):
        kmeans = MiniBatchKMeans(n_clusters=k, random_state=0, verbose=False, n_init=1, max_iter=200, batch_size=10000)
        kmeans.fit(data)
        return kmeans.cluster_centers_
    
    def extract_patches(data):
        patches = []
        reshaped = data.reshape(-1,32,32,3)
        n = int(patch_num / (32-PATCH_SIZE+1) ** 2 + 1)
        for i in tqdm(range(n)):
            for r in range(32-PATCH_SIZE+1):
                for c in range(32-PATCH_SIZE+1):
                    patch = reshaped[i][c:(c+PATCH_SIZE),r:(r+PATCH_SIZE)].flatten()
                    patch_norm = normalize_patch(patch, eps=10)
                    patches.append(patch_norm)

        P = np.vstack(patches)
        return whiten(P)
    
    def dist(x,y):
        return np.sqrt((x - y).dot(x-y))

    def create_patch_features(X):    
        X_mapped_list_per_image = []
        for i in tqdm(range(X.shape[0])):
            mapped_features = []
            for r in range(0, 32-PATCH_SIZE+1, STRIDE):
                for c in range(0, 32-PATCH_SIZE+1, STRIDE):
                    patch = X[i].reshape(32,32,3)[c:(c+PATCH_SIZE),r:(r+PATCH_SIZE)].flatten()
                    patch_norm = normalize_patch(patch, eps=0.01)
                    mapped_features.append([dist(patch_norm, f) for f in filters_final])
            X_mapped_list_per_image.append(np.vstack(mapped_features))
        X_mapped = np.asarray(X_mapped_list_per_image).reshape(-1, ((32-PATCH_SIZE)//STRIDE+1)**2*filters_final.shape[0])
        return X_mapped

    def create_patch_features__vectorized(X):    
        X_mapped_list_per_image = []
        for i in tqdm(range(X.shape[0])):
            patches = image.extract_patches_2d(X[i], (PATCH_SIZE, PATCH_SIZE))
            strided_patches = patches.reshape( 32-PATCH_SIZE+1 , 32-PATCH_SIZE+1, PATCH_SIZE, PATCH_SIZE, 3)[::STRIDE,::STRIDE,:,:,:]
            strided_patches = strided_patches.reshape(((32-PATCH_SIZE)//STRIDE+1)**2, PATCH_SIZE * PATCH_SIZE * 3)
            mapped_features = euclidean_distances(np.asarray([normalize_patch(patch, eps=0.01) for patch in strided_patches]), filters_final)
            X_mapped_list_per_image.append(mapped_features.reshape(((32-PATCH_SIZE)//STRIDE+1)**2 * filters_final.shape[0]))
        X_mapped = np.asarray(X_mapped_list_per_image)
        return X_mapped

    
    def fit(X_trn, y_trn):
        P_zca = extract_patches(X_trn)
        filters_final = run_cached(centroids, P_zca, 'kmeans')
        X_mapped_trn = run_cached(create_patch_features__vectorized, X_trn, 'X_mapped_trn')
        

In [25]:
def contrast(image):
    return (image-image.min())/(image.max() - image.min())

def normalize_patch(patch, eps=10):
    return (patch - patch.mean())/np.sqrt(patch.var() + eps)

def whiten(X):
    X_norm = (X - X.mean(axis=0))/X.std(axis=0)
    cov = np.cov(X_norm, rowvar=False) 
    U,S,V = np.linalg.svd(cov)

    X_zca = U.dot(np.diag(1.0/np.sqrt(S + 0.1))).dot(U.T).dot(X_norm.T).T
    return X_zca

In [26]:
def run_cached(fun, data, name):
    return fun(data)

In [27]:
def extract_patches(data):
    patches = []
    reshaped = data.reshape(-1,32,32,3)
    n = int(patch_num / (32-PATCH_SIZE+1) ** 2 + 1)
    for i in tqdm(range(n)):
        for r in range(32-PATCH_SIZE+1):
            for c in range(32-PATCH_SIZE+1):
                patch = reshaped[i][c:(c+PATCH_SIZE),r:(r+PATCH_SIZE)].flatten()
                patch_norm = normalize_patch(patch, eps=10)
                patches.append(patch_norm)

    P = np.vstack(patches)
    return whiten(P)

P_zca = run_cached(extract_patches, X_trn, 'P_zca')

  0%|          | 0/138 [00:00<?, ?it/s]

# Clusters

In [28]:
from sklearn.cluster import KMeans, MiniBatchKMeans
import numpy as np

def centroids(data):
    kmeans = MiniBatchKMeans(n_clusters=k, random_state=0, verbose=False, n_init=1, max_iter=200, batch_size=10000)
    kmeans.fit(data)
    return kmeans.cluster_centers_

filters_final = run_cached(centroids, P_zca, 'kmeans')

# Some intuition what the patches are

In [30]:
def dist(x,y):
    return np.sqrt((x - y).dot(x-y))

def create_patch_features(X):    
    X_mapped_list_per_image = []
    for i in tqdm(range(X.shape[0])):
        mapped_features = []
        for r in range(0, 32-PATCH_SIZE+1, STRIDE):
            for c in range(0, 32-PATCH_SIZE+1, STRIDE):
                patch = X[i].reshape(32,32,3)[c:(c+PATCH_SIZE),r:(r+PATCH_SIZE)].flatten()
                patch_norm = normalize_patch(patch, eps=0.01)
                mapped_features.append([dist(patch_norm, f) for f in filters_final])
        X_mapped_list_per_image.append(np.vstack(mapped_features))
    X_mapped = np.asarray(X_mapped_list_per_image).reshape(-1, ((32-PATCH_SIZE)//STRIDE+1)**2*filters_final.shape[0])
    return X_mapped


In [31]:
from sklearn.feature_extraction import image
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import euclidean_distances

def create_patch_features__vectorized(X):    
    X_mapped_list_per_image = []
    for i in tqdm(range(X.shape[0])):
        patches = image.extract_patches_2d(X[i], (PATCH_SIZE, PATCH_SIZE))
        strided_patches = patches.reshape( 32-PATCH_SIZE+1 , 32-PATCH_SIZE+1, PATCH_SIZE, PATCH_SIZE, 3)[::STRIDE,::STRIDE,:,:,:]
        strided_patches = strided_patches.reshape(((32-PATCH_SIZE)//STRIDE+1)**2, PATCH_SIZE * PATCH_SIZE * 3)
        mapped_features = euclidean_distances(np.asarray([normalize_patch(patch, eps=0.01) for patch in strided_patches]), filters_final)
        X_mapped_list_per_image.append(mapped_features.reshape(((32-PATCH_SIZE)//STRIDE+1)**2 * filters_final.shape[0]))
    X_mapped = np.asarray(X_mapped_list_per_image)
    return X_mapped

In [32]:
X_mapped_trn = run_cached(create_patch_features__vectorized, X_trn, 'X_mapped_trn')

  0%|          | 0/50000 [00:00<?, ?it/s]

In [33]:
X_mapped_tst = run_cached(create_patch_features__vectorized, X_tst, 'X_mapped_tst')

  0%|          | 0/10000 [00:00<?, ?it/s]

# Logistic Regression on mapped features

In [34]:
def normalize(data):
    return (data - data.mean(axis=0))/data.std(axis=0)

In [35]:
#X_mapped_trn_norm = run_cached(normalize, X_mapped_trn, "X_mapped_trn_norm")
#X_mapped_tst_norm = run_cached(normalize, X_mapped_tst, "X_mapped_tst_norm")
X_mapped_trn_norm = normalize(X_mapped_trn)
X_mapped_tst_norm = normalize(X_mapped_tst)

In [37]:
from sklearn.linear_model import LogisticRegression

def logreg(data):
    x, y = data
    clf = LogisticRegression(random_state=0, max_iter=100, n_jobs=-1, verbose=True)
    clf.fit(x, y.flatten())
    return clf

clf = run_cached(logreg, (X_mapped_trn_norm, y_trn), 'logreg')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =        40970     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.15129D+05    |proj g|=  4.73127D+03


 This problem is unconstrained.



At iterate   50    f=  7.41815D+04    |proj g|=  5.11790D+02

At iterate  100    f=  6.52776D+04    |proj g|=  3.59526D+02

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
40970    100    104      1     0     0   3.595D+02   6.528D+04
  F =   65277.633757993317     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 


/opt/homebrew/Caskroom/miniconda/base/envs/ml/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.8min finished


# Results! 😊

In [39]:
y_trn_pred = clf.predict(X_mapped_trn_norm)
print(f"Train: {(y_trn.flatten() == y_trn_pred).mean()}")

y_tst_pred = clf.predict(X_mapped_tst_norm)
print(f"Test:  {(y_tst.flatten() == y_tst_pred).mean()}")


Train: 0.54722
Test:  0.4851
